## Linear Regression

### Added snippet used as a reference for all models

This model is based on the TuringTutorial example [LinearRegression](https://github.com/TuringLang/TuringTutorials/blob/csp/linear/LinearRegression.ipynb) by Cameron Pfiffer.

Turing is powerful when applied to complex hierarchical models, but it can also be put to task at common statistical procedures, like linear regression. This tutorial covers how to implement a linear regression model in Turing.

We begin by importing all the necessary libraries.

In [1]:
using StatisticalRethinking, CmdStan, GLM
gr(size=(600,600))

ProjDir = rel_path("..", "scripts", "00")
cd(ProjDir)

Import the dataset.

In [2]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [3]:
data = filter(row -> row[:age] >= 18, df)

,height,weight,age,male
,Float64,Float64,Float64,Int64
1,151.765,47.8256,63.0,1
2,139.7,36.4858,63.0,0
3,136.525,31.8648,65.0,0
4,156.845,53.0419,41.0,1
5,145.415,41.2769,51.0,0
6,163.83,62.9926,35.0,1
7,149.225,38.2435,32.0,0
8,168.91,55.48,27.0,1
9,147.955,34.8699,19.0,0


Show the first six rows of the dataset.

In [4]:
first(data, 6)

,height,weight,age,male
,Float64,Float64,Float64,Int64
1,151.765,47.8256,63.0,1
2,139.7,36.4858,63.0,0
3,136.525,31.8648,65.0,0
4,156.845,53.0419,41.0,1
5,145.415,41.2769,51.0,0
6,163.83,62.9926,35.0,1


The next step is to get our data ready for testing. We'll split the mtcars dataset into two subsets, one for training our model and one for evaluating our model. Then, we separate the labels we want to learn (MPG, in this case) and standardize the datasets by subtracting each column's means and dividing by the standard deviation of that column.

The resulting data is not very familiar looking, but this standardization process helps the sampler converge far easier. We also create a function called unstandardize, which returns the standardized values to their original form. We will use this function later on when we make predictions.

Split our dataset 70%/30% into training/test sets.

In [5]:
n = size(data, 1)
test_ind = sample(1:n, Int(floor(0.3*n)), replace=false);
train_ind = [(i) for i=1:n if !(i in test_ind)];
test = data[test_ind, :];
train = data[train_ind, :];

Save dataframe versions of our dataset.

In [6]:
train_cut = DataFrame(train)
test_cut = DataFrame(test)

,height,weight,age,male
,Float64,Float64,Float64,Int64
1,154.94,48.2225,26.0,0
2,150.495,43.4598,27.0,0
3,162.56,56.0186,42.0,1
4,163.83,55.9336,35.0,1
5,148.59,39.4342,47.0,0
6,152.4,40.8233,49.0,0
7,147.955,47.4004,36.0,0
8,170.18,47.5988,58.0,1
9,160.655,48.506,24.0,1


Create our labels. These are the values we are trying to predict.

In [7]:
train_label = train[:, :height]
test_label = test[:, :height]

105-element Array{Float64,1}:
 154.94 
 150.495
 162.56 
 163.83 
 148.59 
 152.4  
 147.955
 170.18 
 160.655
 146.05 
   ⋮    
 163.195
 153.035
 167.005
 152.4  
 150.495
 171.45 
 140.335
 163.83 
 160.655

Get the list of columns to keep.

In [8]:
remove_names = filter(x->!in(x, [:height, :age, :male]), names(data))

1-element Array{Symbol,1}:
 :weight

Filter the test and train sets.

In [9]:
train = Matrix(train[:, remove_names]);
test = Matrix(test[:, remove_names]);

A handy helper function to rescale our dataset.

In [10]:
function standardize(x)
    return (x .- mean(x, dims=1)) ./ std(x, dims=1), x
end

standardize (generic function with 1 method)

Another helper function to unstandardize our datasets.

In [11]:
function unstandardize(x, orig)
    return x .* std(orig, dims=1) .+ mean(orig, dims=1)
end

unstandardize (generic function with 1 method)

Standardize our dataset.

In [12]:
(train, train_orig) = standardize(train)
(test, test_orig) = standardize(test)
(train_label, train_l_orig) = standardize(train_label)
(test_label, test_l_orig) = standardize(test_label);

Design matrix

In [13]:
dmat = [ones(size(train, 1)) train]

247×2 Array{Float64,2}:
 1.0  -1.33631 
 1.0  -2.0641  
 1.0   1.27122 
 1.0  -0.581734
 1.0   2.83842 
 1.0  -1.59081 
 1.0   1.49893 
 1.0   0.775611
 1.0  -0.590664
 1.0  -1.40775 
 ⋮             
 1.0  -0.135239
 1.0  -0.148634
 1.0   0.4586  
 1.0  -0.876421
 1.0  -0.617453
 1.0  -1.68904 
 1.0   1.13281 
 1.0   1.43196 
 1.0   1.19085 

Bayesian linear regression.

In [14]:
lrmodel = "
data {
  int N; //the number of observations
  int K; //the number of columns in the model matrix
  real y[N]; //the response
  matrix[N,K] X; //the model matrix
}
parameters {
  vector[K] beta; //the regression parameters
  real sigma; //the standard deviation
}
transformed parameters {
  vector[N] linpred;
  linpred <- X*beta;
}
model {
  beta[1] ~ cauchy(0,10); // prior for the intercept following Gelman 2008

  for(i in 2:K)
   beta[i] ~ cauchy(0,2.5); // prior for the slopes following Gelman 2008

  y ~ normal(linpred,sigma);
}
";

Define the Stanmodel and set the output format to :mcmcchains.

In [15]:
stanmodel = Stanmodel(name="linear_regression",
  model=lrmodel, output_format=:mcmcchains);
# Input data for cmdstan
lrdata = Dict("N" => size(train, 1), "K" => size(dmat, 2), "y" => train_label, "X" => dmat);


File /Users/rob/.julia/dev/StatisticalRethinking/scripts/00/tmp/linear_regression.stan will be updated.



Sample using cmdstan

In [16]:
rc, chain, cnames = stan(stanmodel, lrdata, ProjDir, diagnostics=false,
  summary=false, CmdStanDir=CMDSTAN_HOME);
# Convert to a  Chain object
chns = set_section(chain, Dict(
    :parameters => ["beta.1", "beta.2", "sigma"],
    :linpred => ["linpred.$i" for i in 1:247],
    :internals => ["lp__", "accept_stat__", "stepsize__", "treedepth__",
      "n_leapfrog__", "divergent__", "energy__"]
  )
)

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is -32.6615, but must be > 0!  (in '/Users/rob/.julia/dev/StatisticalRethinking/scripts/00/tmp/linear_regression.stan' at line 21)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is -31.7784, but must be > 0!  (in '/Users/rob/.julia/dev/StatisticalRethinking/scripts/00/tmp/linear_regression.stan' at line 21)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be ei

Object of type Chains, with data of type 1000×257×4 Array{Float64,3}

Iterations        = 1001:2000
Thinning interval = 1
Chains            = 1, 2, 3, 4
Samples per chain = 1000
linpred           = linpred.1, linpred.2, linpred.3, linpred.4, linpred.5, linpred.6, linpred.7, linpred.8, linpred.9, linpred.10, linpred.11, linpred.12, linpred.13, linpred.14, linpred.15, linpred.16, linpred.17, linpred.18, linpred.19, linpred.20, linpred.21, linpred.22, linpred.23, linpred.24, linpred.25, linpred.26, linpred.27, linpred.28, linpred.29, linpred.30, linpred.31, linpred.32, linpred.33, linpred.34, linpred.35, linpred.36, linpred.37, linpred.38, linpred.39, linpred.40, linpred.41, linpred.42, linpred.43, linpred.44, linpred.45, linpred.46, linpred.47, linpred.48, linpred.49, linpred.50, linpred.51, linpred.52, linpred.53, linpred.54, linpred.55, linpred.56, linpred.57, linpred.58, linpred.59, linpred.60, linpred.61, linpred.62, linpred.63, linpred.64, linpred.65, linpred.66, linpred.67, linpred

Describe the chains.

In [17]:
describe(chns)

2-element Array{ChainDataFrame,1}

Summary Statistics
. Omitted printing of 2 columns
│ Row │ parameters │ mean         │ std       │ naive_se    │ mcse        │
│     │ Symbol     │ Float64      │ Float64   │ Float64     │ Float64     │
├─────┼────────────┼──────────────┼───────────┼─────────────┼─────────────┤
│ 1   │ beta.1     │ -0.000867816 │ 0.0433812 │ 0.000685918 │ 0.000645883 │
│ 2   │ beta.2     │ 0.732033     │ 0.0441591 │ 0.000698217 │ 0.000709541 │
│ 3   │ sigma      │ 0.684781     │ 0.0302918 │ 0.000478955 │ 0.000496051 │

Quantiles
. Omitted printing of 1 columns
│ Row │ parameters │ 2.5%       │ 25.0%     │ 50.0%        │ 75.0%     │
│     │ Symbol     │ Float64    │ Float64   │ Float64      │ Float64   │
├─────┼────────────┼────────────┼───────────┼──────────────┼───────────┤
│ 1   │ beta.1     │ -0.0852518 │ -0.029497 │ -0.000865731 │ 0.0283716 │
│ 2   │ beta.2     │ 0.647109   │ 0.701001  │ 0.732075     │ 0.762098  │
│ 3   │ sigma      │ 0.629284   │ 0.66376   │ 0.68

Perform multivariate OLS.

In [18]:
ols = lm(@formula(height ~ weight), train_cut)

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = (::getfield(StatsModels, Symbol("##18#19")){DataFrame})(::Symbol) at modelframe.jl:145
└ @ StatsModels /Users/rob/.julia/packages/StatsModels/pBxdt/src/modelframe.jl:145
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = check_non_redundancy!(::StatsModels.Terms, ::DataFrame) at modelframe.jl:84
└ @ StatsModels /Users/rob/.julia/packages/StatsModels/pBxdt/src/modelframe.jl:84
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = #modelmat_cols#30(::Bool, ::typeof(StatsModels.modelmat_cols), ::Type{Array{Float64,2}}, ::Symbol, ::StatsModels.ModelFrame) at modelmatrix.jl:34
└ @ StatsModels /Users/rob/.julia/packages/StatsModels/pBxdt/src/modelmatrix.jl:34
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind

StatsModels.DataFrameRegressionModel{GLM.LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

Formula: height ~ 1 + weight

Coefficients:
─────────────────────────────────────────────────────────────────────────────
               Estimate  Std. Error  t value  Pr(>|t|)   Lower 95%  Upper 95%
─────────────────────────────────────────────────────────────────────────────
(Intercept)  114.943      2.37309    48.4361    <1e-99  110.269     119.617  
weight         0.881756   0.0522538  16.8745    <1e-42    0.778832    0.98468
─────────────────────────────────────────────────────────────────────────────

Store our predictions in the original dataframe.

In [19]:
train_cut.OLSPrediction = predict(ols);
test_cut.OLSPrediction = predict(ols, test_cut);

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = (::getfield(StatsModels, Symbol("##18#19")){DataFrame})(::Symbol) at modelframe.jl:145
└ @ StatsModels /Users/rob/.julia/packages/StatsModels/pBxdt/src/modelframe.jl:145
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = check_non_redundancy!(::StatsModels.Terms, ::DataFrame) at modelframe.jl:84
└ @ StatsModels /Users/rob/.julia/packages/StatsModels/pBxdt/src/modelframe.jl:84
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = #modelmat_cols#30(::Bool, ::typeof(StatsModels.modelmat_cols), ::Type{Array{Float64,2}}, ::Symbol, ::StatsModels.ModelFrame) at modelmatrix.jl:34
└ @ StatsModels /Users/rob/.julia/packages/StatsModels/pBxdt/src/modelmatrix.jl:34


Make a prediction given an input vector.

In [20]:
function prediction(chn, x)
    α = Array(chn[Symbol("beta.1")]);
    β = Array(chn[Symbol("beta.2")]);
    return  mean(α) .+ x .* mean(β)
end

prediction (generic function with 1 method)

Calculate the predictions for the training and testing sets.

In [21]:
train_cut.BayesPredictions = unstandardize(prediction(chns, train), train_l_orig)[:,1];
test_cut.BayesPredictions = unstandardize(prediction(chns, test), test_l_orig)[:,1];

Show the first side rows of the modified dataframe.

In [22]:
remove_names = filter(x->!in(x, [:age, :male]), names(test_cut));
test_cut = test_cut[remove_names];
first(test_cut, 6)

bayes_loss1 = sum((train_cut.BayesPredictions - train_cut.height).^2);
ols_loss1 = sum((train_cut.OLSPrediction - train_cut.height).^2);

bayes_loss2 = sum((test_cut.BayesPredictions - test_cut.height).^2);
ols_loss2 = sum((test_cut.OLSPrediction - test_cut.height).^2);

println("\nTraining set:")
println("  Bayes loss: $bayes_loss1")
println("  OLS loss: $ols_loss1")

println("Test set:")
println("  Bayes loss: $bayes_loss2")
println("  OLS loss: $ols_loss2")

┌ Warning: `getindex(df::DataFrame, col_inds::Union{AbstractVector, Regex, Not})` is deprecated, use `df[:, col_inds]` instead.
│   caller = top-level scope at string:2
└ @ Core string:2

Training set:
  Bayes loss: 6634.316748857877
  OLS loss: 6634.287858481152
Test set:
  Bayes loss: 2435.0662987025767
  OLS loss: 2428.523148149471


Plot the chains.

In [23]:
#plot(chain)

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*